# **1. Library**

In [1]:
import pandas as pd
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 
import nltk
import re
import emoji
from nltk.corpus import stopwords
import math
from collections import Counter
import operator
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns


nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def stem(word):
    katadasar_file = open('kata-dasar.txt', 'r')
    katadasar_list = katadasar_file.read().splitlines()
    katadasar_file.close()

    if word in katadasar_list:
        return word 
    
    if word.endswith('kah') or word.endswith('lah') or word.endswith('pun') or word.endswith('nya') :
        word = word[:-3]
    
    if word.endswith('ku') or word.endswith('mu'):
        word = word[:-2]
        

    prefixes = ['meng', 'meny', 'men', 'mem', 'me', 'peng', 'peny', 'pen', 'pem', 'di', 'ter', 'ke']
    found = False
    for prefix in prefixes:
        if word.startswith(prefix):
            word = word[len(prefix):]
            found = True
            break
    
    if not found:
        second_order_prefixes = ['ber', 'bel', 'be', 'per','pel','pe']
        for prefix in second_order_prefixes:
            if word.startswith(prefix):
                word = word[len(prefix):]
                break
    
    if word.startswith('me'):
        word = word[2:]
    elif word.startswith('pe'):
        word = word[2:]
    
    suffixes = ['kan', 'an']
    found = False
    for suffix in suffixes:
        if word.endswith(suffix):
            word = word[:-len(suffix)]
            found = True
            break
    
    if found:
        second_order_prefixes = ['ber', 'bel', 'be', 'per','pel','pe']
        for prefix in second_order_prefixes:
            if word.startswith(prefix):
                word = word[len(prefix):]
                break
    
    return word

# **2. Data 50/50**

In [3]:
Data = pd.read_csv ('Data5050.csv')
Data

,content,sentiment
0,Aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,"MAHAL, KALAH BERSAING, SELAMAT TINGGAL GRAB, T...",-1
2,Aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,Sistem aplikasi lemot. Keluar sendiri.,-1
4,Saya baru instal ulang apk nya.tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,"Knpa sih hrus diupdate tros, memori guw pnuhh",-1


In [4]:
def preprocessing(tweets):
    tweets = tweets.encode('ascii','ignore').decode('utf-8')
    tweets = re.sub(r'[^\x00-\x7f]',r'',tweets)
    tweets = re.sub(r'[_(){}[]]+','',tweets)
    tweets = re.sub(r'@[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'#[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'_[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'-[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r':[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'https?:\/\/\S+','',tweets)
    tweets = re.sub(r'\d+','',tweets)
    tweets = re.sub(r'@+','',tweets)
    tweets = re.sub(r':+','',tweets)
    tweets = re.sub(r',+','',tweets)
    tweets = re.sub(r'_+','',tweets)
    tweets = tweets.lower()
    
    allemot = [str for str in tweets]
    listemot = [x for x in allemot if x in emoji.UNICODE_EMOJI]
    tweets = ' '.join([str for str in tweets.split()if not any (y in str for str in listemot)])
    return tweets

Data['content'] = Data['content'].apply(preprocessing)
Data

,content,sentiment
0,aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,sistem aplikasi lemot. keluar sendiri.,-1
4,saya baru instal ulang apk nya.tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [5]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

Data['content']  = Data['content'].apply(remove_punctuations)
Data

,content,sentiment
0,aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,saya baru instal ulang apk nya tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [6]:
Data.to_csv('Dataclean5050.csv',index = False)

**Zipf's Law based Frequency**

In [7]:
split_words = Data['content'].str.split()
list_split_words = list(chain(*split_words))

count_word= Counter(list_split_words)
df_ = pd.DataFrame(data=count_word, index=['count'])
most_frequency_words = df_.T.sort_values(by=['count'], ascending=False).reset_index()
wordlist = most_frequency_words[most_frequency_words['count'] >= 10]
wordlist = wordlist['index'].to_list()

with open('Stopword5050.txt', 'w') as file:
    file.write('\n'.join(wordlist))

## **2.1 Nazief-Adriani & The Classical Method**

In [8]:
Data = pd.read_csv('Dataclean5050.csv')
Data

,content,sentiment
0,aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,saya baru instal ulang apk nya tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [9]:
Data.dropna(subset = ['content'],inplace = True)

In [10]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [11]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

text_stopwords = []
for index, row in Data.iterrows():
    text_stopwords.append(stopword.remove(row['content']))

Data['content'] = text_stopwords
Data

,content,sentiment
0,aplikasi bermanfaat bisa melihat lokasi driver...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh paketin tpi gk nemu dr...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,baru instal ulang apk nya pas diminta akses lo...,-1
...,...,...
145,klo ga update dpt driver jauh jauh,-1
146,aplikasi grab membantu banget buat yang ibu ru...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [12]:
mystoplist = open("Bahasa.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,aplikasi bermanfaat lokasi driver online tsb,1
1,mahal bersaing selamat tinggal grab terima men...,-1
2,aplikasi anjing gua udh paketin tpi nemu drive...,-1
3,sistem aplikasi lemot,-1
4,instal ulang akses lokasi gabisa yamalah ngula...,-1
...,...,...
145,klo update dpt driver,-1
146,aplikasi grab membantu banget rumah tangga rep...,1
147,angel angel,-1
148,knpa hrus diupdate tros memori guw pnuhh,-1


In [13]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

text_stemmed = []
for index, row in Data.iterrows():
    text_stemmed.append(stemmer.stem(row['content']))

Data['content'] = text_stemmed
Data

,content,sentiment
0,aplikasi manfaat lokasi driver online tsb,1
1,mahal saing selamat tinggal grab terima tani t...,-1
2,aplikasi anjing gua udh paketin tpi nemu drive...,-1
3,sistem aplikasi lot,-1
4,instal ulang akses lokasi gabisa yamalah ngula...,-1
...,...,...
145,klo update dpt driver,-1
146,aplikasi grab bantu banget rumah tangga repot ...,1
147,angel angel,-1
148,knpa hrus diupdate tros memori guw pnuhh,-1


In [14]:
Data.to_csv('Data21.csv',index = False)

## **2.2 Nazief Adriani & Zipf's Law**

In [15]:
Data = pd.read_csv('Dataclean5050.csv')
Data.dropna(subset = ['content'],inplace = True)
Data

,content,sentiment
0,aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,saya baru instal ulang apk nya tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [16]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [17]:
mystoplist = open("Stopword5050.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,bermanfaat supaya melihat lokasi online tsb,1
1,mahal kalah bersaing selamat tinggal terima ka...,-1
2,anjing gua udh paketin tpi nemu tpi promo ambi...,-1
3,sistem lemot keluar sendiri,-1
4,baru instal ulang apk tapi pas diminta akses l...,-1
...,...,...
145,klo dpt jauh jauh,-1
146,yang ibu rumah tangga repot keluar rumah kalo ...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [18]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

text_stemmed = []
for index, row in Data.iterrows():
    text_stemmed.append(stemmer.stem(row['content']))

Data['content'] = text_stemmed
Data

,content,sentiment
0,manfaat supaya lihat lokasi online tsb,1
1,mahal kalah saing selamat tinggal terima kasih...,-1
2,anjing gua udh paketin tpi nemu tpi promo ambi...,-1
3,sistem lot keluar sendiri,-1
4,baru instal ulang apk tapi pas minta akses lok...,-1
...,...,...
145,klo dpt jauh jauh,-1
146,yang ibu rumah tangga repot keluar rumah kalo ...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [19]:
Data.to_csv('Data22.csv',index = False)

## **2.3 Porter & The Classical Method**

In [20]:
Data = pd.read_csv('Dataclean5050.csv')
Data.dropna(subset = ['content'],inplace = True)
Data

,content,sentiment
0,aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,saya baru instal ulang apk nya tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [21]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [22]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

text_stopwords = []
for index, row in Data.iterrows():
    text_stopwords.append(stopword.remove(row['content']))

Data['content'] = text_stopwords
Data

,content,sentiment
0,aplikasi bermanfaat bisa melihat lokasi driver...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh paketin tpi gk nemu dr...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,baru instal ulang apk nya pas diminta akses lo...,-1
...,...,...
145,klo ga update dpt driver jauh jauh,-1
146,aplikasi grab membantu banget buat yang ibu ru...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [23]:
mystoplist = open("Bahasa.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,aplikasi bermanfaat lokasi driver online tsb,1
1,mahal bersaing selamat tinggal grab terima men...,-1
2,aplikasi anjing gua udh paketin tpi nemu drive...,-1
3,sistem aplikasi lemot,-1
4,instal ulang akses lokasi gabisa yamalah ngula...,-1
...,...,...
145,klo update dpt driver,-1
146,aplikasi grab membantu banget rumah tangga rep...,1
147,angel angel,-1
148,knpa hrus diupdate tros memori guw pnuhh,-1


In [24]:
def stemming_porter(text):
    words = text.split()
    stemmed_words = []
    for word in words:
        stemmed_word = stem(word) 
        stemmed_words.append(stemmed_word)
    return ' '.join(stemmed_words)

Data['content'] = Data['content'].apply(stemming_porter)
Data

,content,sentiment
0,aplikasi manfaat lokasi driver online tsb,1
1,mahal saing selamat tinggal grab terima emani ...,-1
2,aplikasi anjing gua udh paketin tpi ne driver ...,-1
3,sistem aplikasi lemot,-1
4,instal ulang akses lokasi gabisa yama ngulang ...,-1
...,...,...
145,klo update dpt driver,-1
146,aplikasi grab bantu banget rumah tangga repot ...,1
147,angel angel,-1
148,knpa hrus update tros memori guw pnuhh,-1


In [25]:
Data.to_csv('Data23.csv',index = False)

## **2.4 Porter & Zipf's Law**

In [26]:
Data = pd.read_csv('Dataclean5050.csv')
Data.dropna(subset = ['content'],inplace = True)
Data

,content,sentiment
0,aplikasi ini bermanfaat supaya bisa melihat lo...,1
1,mahal kalah bersaing selamat tinggal grab teri...,-1
2,aplikasi anjing gua udh di paketin tpi gk nemu...,-1
3,sistem aplikasi lemot keluar sendiri,-1
4,saya baru instal ulang apk nya tapi pas dimint...,-1
...,...,...
145,klo ga di update dpt driver jauh jauh,-1
146,aplikasi grab ini membantu banget buat saya ya...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [27]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [28]:
mystoplist = open("Stopword5050.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,bermanfaat supaya melihat lokasi online tsb,1
1,mahal kalah bersaing selamat tinggal terima ka...,-1
2,anjing gua udh paketin tpi nemu tpi promo ambi...,-1
3,sistem lemot keluar sendiri,-1
4,baru instal ulang apk tapi pas diminta akses l...,-1
...,...,...
145,klo dpt jauh jauh,-1
146,yang ibu rumah tangga repot keluar rumah kalo ...,1
147,angel angel,-1
148,knpa sih hrus diupdate tros memori guw pnuhh,-1


In [29]:
def stemming_porter(text):
    words = text.split()
    stemmed_words = []
    for word in words:
        stemmed_word = stem(word) 
        stemmed_words.append(stemmed_word)
    return ' '.join(stemmed_words)

Data['content'] = Data['content'].apply(stemming_porter)
Data

,content,sentiment
0,manfaat supaya lihat lokasi online tsb,1
1,mahal kalah saing selamat tinggal terima kasih...,-1
2,anjing gua udh paketin tpi ne tpi promo ambil ...,-1
3,sistem lemot keluar sendiri,-1
4,baru instal ulang apk tapi pas minta akses lok...,-1
...,...,...
145,klo dpt jauh jauh,-1
146,yang ibu rumah tangga repot keluar rumah kalo ...,1
147,angel angel,-1
148,knpa sih hrus update tros memori guw pnuhh,-1


In [30]:
Data.to_csv('Data24.csv',index =False)

# **3. Data 70/30**

In [31]:
Data = pd.read_csv ('Data7030.csv')
Data = Data.dropna(subset=['content'])

,content,sentiment
0,Tambah sini tambah mahal... promo nya percuma ...,-1
1,Agak menurun ð,1
2,"Akun tibaÂ² ditangguhkan, padahal gapernah bik...",-1
3,Saya suka pelayan nya ðð,1
4,mobil Avanza e bisnis,1
...,...,...
145,"Promo gak bisa di gunakan , driver nya sibuk t...",-1
146,Oke ni,1
147,banyak promo diskon...the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [32]:
def preprocessing(tweets):
    tweets = tweets.encode('ascii','ignore').decode('utf-8')
    tweets = re.sub(r'[^\x00-\x7f]',r'',tweets)
    tweets = re.sub(r'[_(){}[]]+','',tweets)
    tweets = re.sub(r'@[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'#[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'_[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'-[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r':[A-Za-z0-9]+','',tweets)
    tweets = re.sub(r'https?:\/\/\S+','',tweets)
    tweets = re.sub(r'\d+','',tweets)
    tweets = re.sub(r'@+','',tweets)
    tweets = re.sub(r':+','',tweets)
    tweets = re.sub(r',+','',tweets)
    tweets = re.sub(r'_+','',tweets)
    tweets = tweets.lower()
    
    allemot = [str for str in tweets]
    listemot = [x for x in allemot if x in emoji.UNICODE_EMOJI]
    tweets = ' '.join([str for str in tweets.split()if not any (y in str for str in listemot)])
    return tweets

Data['content'] = Data['content'].apply(preprocessing)
Data

,content,sentiment
0,tambah sini tambah mahal... promo nya percuma ...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,saya suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak bisa di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon...the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [33]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

Data['content']  = Data['content'].apply(remove_punctuations)
Data

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,saya suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak bisa di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


**Zipf's Law Based Frequency**

In [34]:
split_words = Data['content'].str.split()
list_split_words = list(chain(*split_words))

count_word= Counter(list_split_words)
df_ = pd.DataFrame(data=count_word, index=['count'])
most_frequency_words = df_.T.sort_values(by=['count'], ascending=False).reset_index()
wordlist = most_frequency_words[most_frequency_words['count'] >= 10]
wordlist = wordlist['index'].to_list()

with open('Stopword7030.txt', 'w') as file:
    file.write('\n'.join(wordlist))

In [35]:
Data.to_csv('Dataclean7030.csv',index = False)

## **3.1 Nazief-Adriani & The Classical Method**

In [36]:
Data = pd.read_csv('Dataclean7030.csv')
Data = Data.dropna(subset=['content'])

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,saya suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak bisa di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [37]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [38]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

text_stopwords = []
for index, row in Data.iterrows():
    text_stopwords.append(stopword.remove(row['content']))

Data['content'] = text_stopwords
Data

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [39]:
mystoplist = open("Bahasa.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,mahal promo promo semahal promo tarif mahal,-1
1,menurun,1
2,akun ditangguhkan gapernah bikin order fiktif ...,-1
3,suka pelayan,1
4,mobil avanza bisnis,1
...,...,...
145,promo driver sibuk,-1
146,oke,1
147,promo diskon the best,1
148,sngat menbantu tpi gc normal harga slalu berub...,1


In [40]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

text_stemmed = []
for index, row in Data.iterrows():
    text_stemmed.append(stemmer.stem(row['content']))

Data['content'] = text_stemmed
Data

,content,sentiment
0,mahal promo promo mahal promo tarif mahal,-1
1,turun,1
2,akun tangguh gapernah bikin order fiktif siste...,-1
3,suka layan,1
4,mobil avanza bisnis,1
...,...,...
145,promo driver sibuk,-1
146,oke,1
147,promo diskon the best,1
148,sngat menbantu tpi gc normal harga slalu ubah ...,1


In [41]:
Data.to_csv('Data31.csv',index = False)

## **3.2 Nazief Adriani & Zipf's Law**

In [42]:
Data = pd.read_csv('Dataclean7030.csv')
Data = Data.dropna(subset=['content'])

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,saya suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak bisa di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [43]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [44]:
mystoplist = open("Stopword7030.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,tambah sini tambah mahal promo percuma aja dul...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,suka pelayan,1
4,mobil avanza bisnis,1
...,...,...
145,promo gunakan sibuk terus,-1
146,oke,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal harga slalu be...,1


In [45]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

text_stemmed = []
for index, row in Data.iterrows():
    text_stemmed.append(stemmer.stem(row['content']))

Data['content'] = text_stemmed
Data

,content,sentiment
0,tambah sini tambah mahal promo percuma aja dul...,-1
1,agak turun,1
2,akun tiba tangguh padahal gapernah bikin order...,-1
3,suka layan,1
4,mobil avanza bisnis,1
...,...,...
145,promo guna sibuk terus,-1
146,oke,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal harga slalu ub...,1


In [46]:
Data.to_csv('Data32.csv',index = False)

## **3.3 Porter & The Classical Method**

In [47]:
Data = pd.read_csv('Dataclean7030.csv')
Data = Data.dropna(subset=['content'])

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,saya suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak bisa di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [48]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [49]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

text_stopwords = []
for index, row in Data.iterrows():
    text_stopwords.append(stopword.remove(row['content']))

Data['content'] = text_stopwords
Data

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [50]:
mystoplist = open("Bahasa.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,mahal promo promo semahal promo tarif mahal,-1
1,menurun,1
2,akun ditangguhkan gapernah bikin order fiktif ...,-1
3,suka pelayan,1
4,mobil avanza bisnis,1
...,...,...
145,promo driver sibuk,-1
146,oke,1
147,promo diskon the best,1
148,sngat menbantu tpi gc normal harga slalu berub...,1


In [51]:
def stemming_porter(text):
    words = text.split()
    stemmed_words = []
    for word in words:
        stemmed_word = stem(word) 
        stemmed_words.append(stemmed_word)
    return ' '.join(stemmed_words)

Data['content'] = Data['content'].apply(stemming_porter)
Data

,content,sentiment
0,mahal promo promo semahal promo tarif mahal,-1
1,urun,1
2,akun tangguh gapernah bikin order fiktif siste...,-1
3,suka ay,1
4,mobil avanza bisnis,1
...,...,...
145,promo driver sibuk,-1
146,oke,1
147,promo diskon the st,1
148,sngat bantu tpi gc normal harga slalu ubah ubah,1


In [52]:
Data.to_csv('Data33.csv',index = False)

## **3.4 Porter & Zipf's Law**

In [53]:
Data = pd.read_csv('Dataclean7030.csv')
Data = Data.dropna(subset=['content'])

,content,sentiment
0,tambah sini tambah mahal promo nya percuma ...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,saya suka pelayan nya,1
4,mobil avanza e bisnis,1
...,...,...
145,promo gak bisa di gunakan driver nya sibuk terus,-1
146,oke ni,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal y harga y slal...,1


In [54]:
Data.isna().sum()

content      0
sentiment    0
dtype: int64

In [55]:
mystoplist = open("Stopword7030.txt", "r").read()

mystoplist_result = []
for index,row in Data.iterrows():
    word_token = nltk.word_tokenize(row['content'])
    word_token = [word for word in word_token if not word in mystoplist]
    mystoplist_result.append(" ".join(word_token))

Data['content'] = mystoplist_result
Data

,content,sentiment
0,tambah sini tambah mahal promo percuma aja dul...,-1
1,agak menurun,1
2,akun tiba ditangguhkan padahal gapernah bikin ...,-1
3,suka pelayan,1
4,mobil avanza bisnis,1
...,...,...
145,promo gunakan sibuk terus,-1
146,oke,1
147,banyak promo diskon the best,1
148,sngat menbantu tpi ko gc normal harga slalu be...,1


In [56]:
def stemming_porter(text):
    words = text.split()
    stemmed_words = []
    for word in words:
        stemmed_word = stem(word) 
        stemmed_words.append(stemmed_word)
    return ' '.join(stemmed_words)

Data['content'] = Data['content'].apply(stemming_porter)
Data

,content,sentiment
0,tambah sini tambah mahal promo percuma aja dul...,-1
1,agak urun,1
2,akun tiba tangguh padahal gapernah bikin order...,-1
3,suka ay,1
4,mobil avanza bisnis,1
...,...,...
145,promo guna sibuk terus,-1
146,oke,1
147,banyak promo diskon the st,1
148,sngat bantu tpi ko gc normal harga slalu ubah ...,1


In [57]:
Data.to_csv('Data34.csv',index = False)